## Evaluation Metrics for Classification Models

### 1. Classification model

Let's begin by repeating the preprocessing and modeling we did in notebook 2:

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

Loading the data:

In [6]:
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

Preprocessing:

In [7]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for column in categorical_columns:
    df[column] = df[column].str.lower().str.replace(' ', '_')

df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
df['totalcharges'] = df['totalcharges'].fillna(0)

df['churn'] = (df['churn'] == 'yes').astype(int)

In [40]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train['churn'].values
y_val = df_val['churn'].values
y_test = df_test['churn'].values

#del df_train['churn']
#del df_val['churn']
#del df_test['churn']

In [9]:
numerical_features = ['tenure', 'monthlycharges', 'totalcharges']

categorical_features = ['gender', 'seniorcitizen', 'partner', 'dependents',
    'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity','onlinebackup',
    'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract',
    'paperlessbilling', 'paymentmethod']

Modeling:

In [41]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_features + numerical_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [51]:
val_dict = df_val[categorical_features + numerical_features].to_dict(orient='records')
X_val = dv.fit_transform(val_dict)

y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = y_pred >= 0.5
(y_val == churn_decision).mean()

0.8034066713981547

### 2. Accuracy and dummy model

#### 2.1 Evaluate the model on different threshols

#### 2.2 Checking the accuracy for dummy baselines

### 3. Confusion table

#### 3.1 Different type of errors and correct decisions

#### 3.2 Arranging errors in a table

### 4. Precision and Recall

### 5. ROC Curves

#### 5.1 TPR and FRP

#### 5.2 Random model

#### 5.3 Ideal model

### 6. ROC AUC

#### 6.1 Area under the ROC curve

#### 6.2 Interpretation of AUC

### 7. Cross-Validation

#### 7.1 Evaluating the same model on different subsets of data

#### 7.2 Getting the average prediction and the spread within prediction